In [5]:
import cv2 as cv
import pytesseract
import json
import path
import pandas as pd

In [6]:
pytesseract.pytesseract.tesseract_cmd = '/usr/local/Cellar/tesseract/5.1.0/bin/tesseract'

In [ ]:
BUDD_DIR = '/Users/napatswift/myproject/bkk-2022-budget/bud-images/65075'

In [7]:
def matToText(filename, write=False):
  mat = cv.imread(filename)
  draw_mat = mat.copy()
  gray = cv.cvtColor(mat, cv.COLOR_BGR2GRAY)
  th, threshed = cv.threshold(gray, 70, 255, cv.THRESH_BINARY_INV|cv.THRESH_OTSU)
  kernel = cv.getStructuringElement(cv.MORPH_RECT, (5, 5))
  threshed = cv.dilate(threshed, kernel)
  hist = cv.reduce(threshed, 1, cv.REDUCE_AVG).reshape(-1)

  kernel = cv.getStructuringElement(cv.MORPH_RECT, (10, 5))
  dilated = cv.dilate(threshed, kernel)
  contours, hier  = cv.findContours(dilated, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
  rects = [cv.boundingRect(c) for c in contours]
  rects.sort(key=lambda rect: rect[1])

  h,w = mat.shape[:2]
  stack_y = []
  for ii in range(h-1):
    if (hist[ii] and not hist[ii+1]):
      stack_y.append(ii)
    elif (not hist[ii] and hist[ii+1]):
      stack_y.append(ii)

  ocr_text = []
  while(stack_y):
    y1 = stack_y.pop()
    y2 = stack_y.pop()
    line = []
    while rects:
      rect = rects[-1]
      midPoint = rect[1] + rect[3]//2
      if (y2 <=  midPoint <= y1):
        cv.rectangle(draw_mat, (rect[0],rect[1]), (rect[0]+rect[2],rect[1]+rect[3]), (0, 0, 255));
        line.append(rects.pop())
      else:
        break
    line.sort(key=lambda rect: rect[0]+rect[2]/2)
    ocr_text.append(
      [
        {
         'text': pytesseract.image_to_string(mat[y:y+h, x:x+w], lang='tha+eng'),
         'x': x,
         'y':y,
         'w':w,
         'h':h,
         'filename': filepath.name
         }
        for x,y,w,h in line
      ]
    )

  if (write):
    filename = filename[filename.rfind('/')+1:filename.rfind('.')]
    cv.imwrite(f"{filename}drawed.jpg", draw_mat)
    cv.imwrite(f"{filename}dilated.jpg", dilated)
    cv.imwrite(f"{filename}threshed.jpg", threshed)

  return ocr_text

In [8]:
1/0

ZeroDivisionError: division by zero

In [ ]:
text_list = []
for ii in range(27, 63):
  filepath = path.Path(f"./bud-images/65075-{ii:03d}.jpg")
  text = matToText(filepath)
  text_list.append(text)

In [ ]:
df_text = []
for page in text_list:
  for line in page[::-1]:
    line_num = 0
    for block in line:
      block['text'] = block['text'].replace('\n','')
      block['line_num'] = line_num
      df_text.append(block)
      line_num += 1


In [ ]:
df = pd.DataFrame(df_text)
df.to_csv("65075.csv", index=0)
df

,text,x,y,w,h,line_num,filename
0,446,761,116,49,25,31,65075-027.jpg
1,รายละเอียดงบประมาณจําแนกตามงบรายจ่าย,120,294,452,32,30,65075-027.jpg
2,งาน: อํานวยการและบริหารสํานักงานเขต,121,347,405,32,29,65075-027.jpg
3,"10,393,960",1230,353,127,28,29,65075-027.jpg
4,บาท,1399,346,55,24,29,65075-027.jpg
...,...,...,...,...,...,...,...
2345,บาท,1401,773,51,22,1,65075-062.jpg
2346,,240,823,95,26,0,65075-062.jpg
2347,(8) ค่าใช้จ่ายโครงการเกษตรปลอดสารพิษ,375,817,396,33,0,65075-062.jpg
2348,"15,000",1278,823,79,29,0,65075-062.jpg


In [ ]:
currImageFilename = df.loc[0].filename
currImage = cv.imread('./bud-images/' + currImageFilename)
for row in df.iterrows():
  if currImageFilename != row[1].filename:
    cv.imwrite('./bud-images/drew/' + currImageFilename, currImage)
    currImageFilename = row[1].filename
    currImage = cv.imread('./bud-images/' + currImageFilename)
  cv.rectangle(currImage, (row[1].x, row[1].y), (row[1].x+row[1].w, row[1].y+row[1].h), (0, 0, 255))
  cv.putText(currImage, f"{row[1].x}, {row[1].y}", (row[1].x, row[1].y), 0, .5, (0, 0, 255))
cv.imwrite('./bud-images/drew/' + currImageFilename, currImage)